# Patient Data Analysis

In [1]:
# import libraries
import pandas as pd
from pathlib import Path
import numpy as np
import os
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt


import sys
sys.path.append(r"/Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/Brain_Mets_Classification")
import brain_mets_classification.config as config

### load data

load excel sheet

In [2]:
# reads the patient excel sheet and puts them into a pandas Dataframe
path_to_excel = Path("/Users/LennartPhilipp/Desktop/Uni/Prowiss/Dateien/patient data analysis correction 31_07_24.xlsx")
try:
    patientsDataFrame = pd.read_excel(path_to_excel)
except ImportError as e:
    print(e)

modifiedPatientsDataFrame = patientsDataFrame.copy()

load patients actually in dataset

In [3]:
#path_to_list_of_patients_in_dataset = Path("/Users/LennartPhilipp/Desktop/Uni/Prowiss/Dateien/dataset_patients_05_24.tsv")
path_to_list_of_patients_in_dataset = Path("/Users/LennartPhilipp/Desktop/Uni/Prowiss/Dateien/participants_01_08_24.tsv")

dataset_patients = pd.read_csv(path_to_list_of_patients_in_dataset, sep="\t", index_col=False)

In [4]:
path_to_dataset_patients = Path("/Users/LennartPhilipp/Desktop/Uni/Prowiss/Dateien/participants.tsv")
path_to_preprocessed_images = Path("/Volumes/BrainMets/Rgb_Brain_Mets/brain_mets_classification/derivatives/preprocessed_brainlesion_allpatients")

#dataset_patients = pd.read_csv(path_to_dataset_patients, sep="\t", index_col=False)

# drop patient sub-01383503
patient_to_drop_index = dataset_patients.index[dataset_patients["participant_id"] == "sub-01383503"]
dataset_patients.drop(index=patient_to_drop_index, inplace=True)
dataset_patients.reset_index(drop=True, inplace=True)
#training_patients = training_patients.sample(frac=1) #shuffles the dataset

print(f"Before removing unfit files: {len(dataset_patients)}")
patient_files_list = os.listdir(path_to_preprocessed_images)
for index, row in dataset_patients.iterrows():
    if dataset_patients["participant_id"][index] not in patient_files_list:
        dataset_patients.drop(index=index, inplace=True)

print(f"After removing unfit files: {len(dataset_patients)}")
dataset_patients.reset_index(drop=True, inplace=True)

Before removing unfit files: 473
After removing unfit files: 471


### DON'T EXECUTE CASUALLY!

In [5]:
# export dataset_patients
#path_to_list_of_patients_in_dataset = Path("/Users/LennartPhilipp/Desktop/Uni/Prowiss/Dateien/dataset_patients_05_24.tsv")
#dataset_patients.to_csv(path_to_list_of_patients_in_dataset, index=False, sep="\t")

### Data clean up

In [6]:
# removes unused columns from dataset
modifiedPatientsDataFrame = modifiedPatientsDataFrame.iloc[:, :50] # only keeps the columns until column 38
modifiedPatientsDataFrame = modifiedPatientsDataFrame.drop(columns=["T.1",
                                                                    "primary_coded",
                                                                    "primary_coded_clean",
                                                                    "N",
                                                                    "M",
                                                                    "M_number_organ_systems",
                                                                    "TNM",
                                                                    "Survival ab ED gesamt",
                                                                    "Survival ab ED Hirnmet.",
                                                                    "Survival_from_ED_Met (months)",
                                                                    "Alter bei MRT (J./kalkuliert)",
                                                                    "Dauer Tumorleiden\n(kalkuliert; von ED bis ED Hirnmet.)",
                                                                    "Karnofsky (%)",
                                                                    "periphere Metastasen zum Zeitpunkt des MRTs ( Keine = 0, Einzelne = 1, Oligo =2, Multipel =3)",
                                                                    "im Verlauf neu aufgetretene Metastasen (0 = keine bekannt)",
                                                                    ])

# turn ID strings into ints
modifiedPatientsDataFrame.ID = modifiedPatientsDataFrame.ID.astype(int)

# drop CUP patients
#modifiedPatientsDataFrame = modifiedPatientsDataFrame.drop(modifiedPatientsDataFrame[modifiedPatientsDataFrame.primary_coded_newv == "29"].index)

dataset_patients["participant_id"] = dataset_patients["participant_id"].str.slice_replace(0, 4, "").astype(int)

### patients selected by whom

In [7]:
print(modifiedPatientsDataFrame["selected by"].value_counts())

selected by
Lennart        483
not Lennart    262
Name: count, dtype: int64


#### patients not selected by Lennart
to do:
- put "MRI sequences missing" into the "reason" column in excel based on unused_pats_not_Lennart

In [8]:
# compare patients not selected by Lennart with dataset_patients
pats_not_used_not_Lennart = []
#pats_not_by_Lennart = modifiedPatientsDataFrame.query("`selected by` == not Lennart")["ID"]
pats_not_by_Lennart = list(modifiedPatientsDataFrame[modifiedPatientsDataFrame["selected by"] == "not Lennart"]["ID"])

print(f"# of patients not selected by Lennart: {len(pats_not_by_Lennart)}")

#print(len(list(dataset_patients["participant_id"])))

amount_of_patients_used_not_lennart = 0

for pat_id in pats_not_by_Lennart:
    if pat_id not in list(dataset_patients["participant_id"]):
        pats_not_used_not_Lennart.append(pat_id)
        amount_of_patients_used_not_lennart += 1

unused_pats_not_Lennart = modifiedPatientsDataFrame[modifiedPatientsDataFrame.ID.isin(pats_not_used_not_Lennart)]

print(f"# of those not used for the dataset: {amount_of_patients_used_not_lennart}")
print("\ndue to the following reasons:")

unused_value_counts = unused_pats_not_Lennart.reason.value_counts()
print(unused_value_counts)
amount_of_patients_not_lennart_not_used_with_reasons = 0
for value in unused_value_counts:
    amount_of_patients_not_lennart_not_used_with_reasons += value

amount_other_reason_not_lennart = amount_of_patients_used_not_lennart - amount_of_patients_not_lennart_not_used_with_reasons
print(f"not correct amount of MRI sequences: {amount_other_reason_not_lennart}")


# of patients not selected by Lennart: 262
# of those not used for the dataset: 46

due to the following reasons:
reason
CUP                    12
braintumor              2
lymphoma                1
no pre op images        1
no brain metastases     1
post resection          1
Name: count, dtype: int64
not correct amount of MRI sequences: 28


#### patients selected by Lennart

In [9]:
# get all patients selected by Lennart
# compare with patients actually used
# list reasons why not used
# get funky

pats_by_Lennart = list(modifiedPatientsDataFrame[modifiedPatientsDataFrame["selected by"] == "Lennart"]["ID"])
pats_not_used_Lennart = []

print(f"# of patients selected by Lennart: {len(pats_by_Lennart)}")

amount_of_patients_used_lennart = 0

for pat_id in pats_by_Lennart:
    if pat_id not in list(dataset_patients["participant_id"]):
        pats_not_used_Lennart.append(pat_id)
        amount_of_patients_used_lennart += 1

unused_pats_Lennart = modifiedPatientsDataFrame[modifiedPatientsDataFrame.ID.isin(pats_not_used_Lennart)]

print(f"# of those not used for the dataset: {amount_of_patients_used_lennart}")
print("\ndue to the following reasons:")

unused_value_counts = unused_pats_Lennart.reason.value_counts()
print(unused_value_counts)
amount_of_patients_lennart_not_used_with_reasons = 0
for value in unused_value_counts:
    amount_of_patients_lennart_not_used_with_reasons += value

amount_other_reason_lennart = amount_of_patients_used_lennart - amount_of_patients_lennart_not_used_with_reasons

print()
print(f"not correct amount of MRI sequences: {amount_other_reason_lennart}")


# of patients selected by Lennart: 483
# of those not used for the dataset: 228

due to the following reasons:
reason
no brain metastases            61
brain tumor                    32
lack of data                   23
pre OP MRI missing             19
lymphoma                       17
histology unclear               8
CUP                             7
other tumor                     3
meningeosis carcinomatosa       2
braintumor                      1
contact to sinus sagittalis     1
dural metastasis                1
Name: count, dtype: int64

not correct amount of MRI sequences: 53


acceptable reasons:\
no brain metastastes, brain tumor, other tumor, lymphoma, pre OP MRI missing, meningeosis carcinomatosa, CUP, missing MRI sequences, lack of data, histology unclear

## Data Analysis

To-do:
- sex
- age at MRI
- age at first diagnosis
- age at first diagnosis brainmet
- did any of the following occur during their therapy: OP / Chemo / Radiatio / Immunotherapie
- height
- weight
- ECOG
- status of resection
- status of relapse

Try to display as such:

CATEGORY\
total n=XXX, mean m: XX (n=XXX) ± STD, mean f: XX (n=XXX) ± STD

In [10]:
dataset_pat_ids = list(dataset_patients["participant_id"])
excel_dataset_pats = modifiedPatientsDataFrame.copy()[modifiedPatientsDataFrame.ID.isin(dataset_pat_ids)]
#print(excel_dataset_pats)
excel_dataset_pats[excel_dataset_pats.duplicated(subset="ID")]

,on_server,"in_study (0 = no, 1 = yes, 2 = tbd, 3 = remarkable)",reason,comment,selected by,MRI_comments,ID,birthdate,sex (m/w),preop_MRICE_date,...,im Verlauf Rezidiv der Hirnmetastase (0 = nicht bekannt; 1 = ja),CRP,Albumin,Kreatinin,eGFR,Hb,Leukozyten,Thrombozyten,Quick,Hirnvolumen


removed 2nd instance of `1349100` in excel sheet\
removed 1st instance of `1935938` in excel sheet\
removed 1st instance of `1699419` in excel sheet\
removed 2nd instance of `1876862` in excel sheet\
removed 1st instance of `1764802` in excel sheet\
removed 1st instance of `1456719` in excel sheet\
removed 1st instance of `2014467` in excel sheet\
removed 1st instance of `1792136` in excel sheet

In [11]:
# get list of patIDs of men and women in dataset
men_ids = list(excel_dataset_pats[excel_dataset_pats["sex (m/w)"] == "m"].ID)
women_ids = list(excel_dataset_pats[excel_dataset_pats["sex (m/w)"] == "w"].ID)

### sex

In [12]:
def percentage(part, whole):
  Percentage = float(part)/float(whole)
  return Percentage

pat_count = len(excel_dataset_pats)
for val, cnt in excel_dataset_pats["sex (m/w)"].value_counts().items():
    print("{:1s}: {:3d}/{:3d} ({:.2%})".format(val, cnt, pat_count, percentage(cnt, pat_count)))

m: 255/471 (54.14%)
w: 216/471 (45.86%)


### age at mri

In [13]:
ages_men = dataset_patients.age[dataset_patients.participant_id.isin(men_ids)]
mean_ages_men = np.mean(ages_men)
median_ages_men = np.median(ages_men)
std_ages_men = np.std(ages_men)

ages_women = dataset_patients.age[dataset_patients.participant_id.isin(women_ids)]
mean_ages_women = np.mean(ages_women)
median_ages_women = np.median(ages_women)
std_ages_women = np.std(ages_women)

print("{:^20}".format("ages men"))
print("{:12s}={:>6d}".format("total: n", len(ages_men)))
print("{:12s}={:>9.2f}".format("mean: m", mean_ages_men))
print("{:12s}={:>9.2f}".format("median: med", median_ages_men))
print("{:12s}={:>9.2f}".format("std", std_ages_men))
print()

print("{:^20}".format("ages women"))
print("{:12s}={:>6d}".format("total: n", len(ages_women)))
print("{:12s}={:>9.2f}".format("mean: m", mean_ages_women))
print("{:12s}={:>9.2f}".format("median: med", median_ages_women))
print("{:12s}={:>9.2f}".format("std", std_ages_women))

      ages men      
total: n    =   255
mean: m     =    63.69
median: med =    65.00
std         =    11.62

     ages women     
total: n    =   216
mean: m     =    60.58
median: med =    61.00
std         =    12.08


### primaries count

#### total

In [14]:
primaries_total = dataset_patients.primary[dataset_patients.participant_id.isin(excel_dataset_pats.ID)]
#primaries_total = dataset_patients.primary
print("{:^44}".format("primaries total - detailed"))

print(f"total: {len(primaries_total)}")

for val, cnt in primaries_total.value_counts().items():
    primary_name = config.cancer_primaries_dict[val]
    print("{:38s}{:>6d} ({:.2%})".format(primary_name, cnt, percentage(cnt, len(primaries_total))))

         primaries total - detailed         
total: 471
adenocarcinoma of the lung               117 (24.84%)
melanoma                                  74 (15.71%)
small cell lung cancer                    58 (12.31%)
breast cancer                             40 (8.49%)
NSCLC                                     29 (6.16%)
colorectal cancer                         23 (4.88%)
squamous cell lung cancer                 20 (4.25%)
lung cancer, exact histology unknown      17 (3.61%)
renal cell carcinoma                      13 (2.76%)
lung cancer, other                        11 (2.34%)
prostate cancer                            8 (1.70%)
transitional cell carcinoma                7 (1.49%)
endometrial cancer                         4 (0.85%)
sarcoma, other                             4 (0.85%)
oral squamous cell carcinoma               4 (0.85%)
esophageal cancer                          4 (0.85%)
testicular cancer                          4 (0.85%)
ovarian cell carcinoma                  

#### men

In [15]:
primaries_men = dataset_patients.primary[dataset_patients.participant_id.isin(men_ids)]
print("{:^44}".format("primaries men - detailed"))

print(f"total: {len(primaries_men)}")



for val, cnt in primaries_men.value_counts().items():
    primary_name = config.cancer_primaries_dict[val]
    print("{:38s}{:>6d} ({:.2%})".format(primary_name, cnt, percentage(cnt, len(primaries_men))))

          primaries men - detailed          
total: 255
adenocarcinoma of the lung                64 (25.10%)
melanoma                                  48 (18.82%)
small cell lung cancer                    34 (13.33%)
colorectal cancer                         15 (5.88%)
lung cancer, exact histology unknown      13 (5.10%)
NSCLC                                     13 (5.10%)
squamous cell lung cancer                 12 (4.71%)
renal cell carcinoma                       8 (3.14%)
prostate cancer                            8 (3.14%)
lung cancer, other                         7 (2.75%)
transitional cell carcinoma                7 (2.75%)
testicular cancer                          4 (1.57%)
esophageal cancer                          4 (1.57%)
oral squamous cell carcinoma               3 (1.18%)
large cell lung cancer                     2 (0.78%)
osteosarcoma                               2 (0.78%)
laryngeal cancer                           2 (0.78%)
rhabdomyosarcoma                        

In [16]:
# find patients with cancers of unknown primareis, shouldn't be in the dataset
# fuck
cup_patients = excel_dataset_pats[excel_dataset_pats.primary_coded_newv == "29"]
cup_patients

,on_server,"in_study (0 = no, 1 = yes, 2 = tbd, 3 = remarkable)",reason,comment,selected by,MRI_comments,ID,birthdate,sex (m/w),preop_MRICE_date,...,im Verlauf Rezidiv der Hirnmetastase (0 = nicht bekannt; 1 = ja),CRP,Albumin,Kreatinin,eGFR,Hb,Leukozyten,Thrombozyten,Quick,Hirnvolumen


#### women

In [17]:
primaries_women = dataset_patients.primary[dataset_patients.participant_id.isin(women_ids)]
print("{:^44}".format("primaries women - detailed"))

print(f"total: {len(primaries_women)}")


for val, cnt in primaries_women.value_counts().items():
    primary_name = config.cancer_primaries_dict[val]
    print("{:38s}{:>6d} ({:.2%})".format(primary_name, cnt, percentage(cnt, len(primaries_women))))

         primaries women - detailed         
total: 216
adenocarcinoma of the lung                53 (24.54%)
breast cancer                             39 (18.06%)
melanoma                                  26 (12.04%)
small cell lung cancer                    24 (11.11%)
NSCLC                                     16 (7.41%)
colorectal cancer                          8 (3.70%)
squamous cell lung cancer                  8 (3.70%)
renal cell carcinoma                       5 (2.31%)
lung cancer, other                         4 (1.85%)
lung cancer, exact histology unknown       4 (1.85%)
ovarian cell carcinoma                     4 (1.85%)
endometrial cancer                         4 (1.85%)
sarcoma, other                             3 (1.39%)
cervical cancer                            2 (0.93%)
gastric cancer                             2 (0.93%)
leiomyosarcoma                             2 (0.93%)
salivary gland cancer                      2 (0.93%)
thyroid cancer                         

### age at first diagnosis tumor

In [18]:
# get values of first diagnosis
# find difficult values
# decide wether to ignore or not
# compute age
# print age

months = {
    "Januar ":"01.01.",
    "Februar ":"01.02.",
    "März ":"01.03.",
    "April ":"01.04.",
    "Mai ":"01.05.",
    "Juni ":"01.06.",
    "Juli ":"01.07.",
    "August ":"01.08.",
    "September ":"01.09.",
    "Oktober ":"01.10.",
    "November ":"01.11.",
    "Dezember ":"01.12.",
    "Jan ":"01.01.",
    "Feb ":"01.02.",
    "Apr ":"01.03.",
    "Jun ":"01.06.",
    "Jul ":"01.07.",
    "Aug ":"01.08.",
    "Sept ":"01.09.",
    "Sep ":"01.09.",
    "Okt ":"01.10.",
    "Nov ":"01.11.",
    "Dez ":"01.12."
}

excel_dataset_pats.date_first_diagnosis_primary = excel_dataset_pats.date_first_diagnosis_primary.astype("str")

for month in months.keys():
    excel_dataset_pats.date_first_diagnosis_primary = excel_dataset_pats.date_first_diagnosis_primary.str.replace(month, months[month])


print(excel_dataset_pats.date_first_diagnosis_primary)

#replace months with numbers
# excel_dataset_pats.date_first_diagnosis_primary = excel_dataset_pats.date_first_diagnosis_primary.str.replace("Januar", "01")

#print(excel_dataset_pats.date_first_diagnosis_primary)

excel_dataset_pats.date_first_diagnosis_primary = pd.to_datetime(excel_dataset_pats.date_first_diagnosis_primary, errors="raise", format="mixed")
excel_dataset_pats.birthdate = pd.to_datetime(excel_dataset_pats.birthdate, errors="raise", format="mixed")


0      2003-09-01 00:00:00
1      2005-04-01 00:00:00
2      2004-09-01 00:00:00
4      2005-05-01 00:00:00
5      2003-01-01 00:00:00
              ...         
735    2023-10-01 00:00:00
736    2021-01-01 00:00:00
740    2023-10-01 00:00:00
741    2023-02-01 00:00:00
744    2023-10-01 00:00:00
Name: date_first_diagnosis_primary, Length: 471, dtype: object


#### men

In [19]:
men_age_first_diagnosis_dict = {}
for index, row in excel_dataset_pats[excel_dataset_pats.ID.isin(men_ids)].iterrows():
    id = row["ID"]

    # calculate the age of the patient when first diagnosed with tumor
    if not pd.isnull(row.date_first_diagnosis_primary):
        age = relativedelta(row.date_first_diagnosis_primary, row.birthdate).years
        men_age_first_diagnosis_dict[id] = age
    else:
        #print(id)
        pass


men_age_first_diagnosis_dict = list(men_age_first_diagnosis_dict.values())

mean_ages_first_diagnosis_men = np.mean(men_age_first_diagnosis_dict)
median_ages_first_diagnosis_men = np.median(men_age_first_diagnosis_dict)
std_ages_first_diagnosis_men = np.std(men_age_first_diagnosis_dict)

print("{:^50}".format("ages of men first diagnosed with the tumor"))
print("{:12s}={:>6d}".format("total: n", len(men_age_first_diagnosis_dict)))
print("{:12s}={:>9.2f}".format("mean: m", mean_ages_first_diagnosis_men))
print("{:12s}={:>9.2f}".format("median: med", median_ages_first_diagnosis_men))
print("{:12s}={:>9.2f}".format("std", std_ages_first_diagnosis_men))


    ages of men first diagnosed with the tumor    
total: n    =   250
mean: m     =    60.87
median: med =    63.00
std         =    14.83


#### women

In [20]:
women_age_first_diagnosis_dict = {}
for index, row in excel_dataset_pats[excel_dataset_pats.ID.isin(women_ids)].iterrows():
    id = row["ID"]

    # calculate the age of the patient when first diagnosed with tumor
    if not pd.isnull(row.date_first_diagnosis_primary):
        age = relativedelta(row.date_first_diagnosis_primary, row.birthdate).years
        women_age_first_diagnosis_dict[id] = age


women_age_first_diagnosis_dict = list(women_age_first_diagnosis_dict.values())

mean_ages_first_diagnosis_women = np.mean(women_age_first_diagnosis_dict)
median_ages_first_diagnosis_women = np.median(women_age_first_diagnosis_dict)
std_ages_first_diagnosis_women = np.std(women_age_first_diagnosis_dict)

print("{:^50}".format("ages of women first diagnosed with the tumor"))
print("{:12s}={:>6d}".format("total: n", len(women_age_first_diagnosis_dict)))
print("{:12s}={:>9.2f}".format("mean: m", mean_ages_first_diagnosis_women))
print("{:12s}={:>9.2f}".format("median: med", median_ages_first_diagnosis_women))
print("{:12s}={:>9.2f}".format("std", std_ages_first_diagnosis_women))

   ages of women first diagnosed with the tumor   
total: n    =   211
mean: m     =    55.45
median: med =    58.00
std         =    19.49


### age at first diagnosis brainmet

In [21]:
# get values of first diagnosis brainmet
# find difficult values
# decide wether to ignore or not
# compute age
# print age

months = {
    "Januar ":"01.01.",
    "Februar ":"01.02.",
    "März ":"01.03.",
    "April ":"01.04.",
    "Mai ":"01.05.",
    "Juni ":"01.06.",
    "Juli ":"01.07.",
    "August ":"01.08.",
    "September ":"01.09.",
    "Oktober ":"01.10.",
    "November ":"01.11.",
    "Dezember ":"01.12.",
    "Jan ":"01.01.",
    "Feb ":"01.02.",
    "Apr ":"01.03.",
    "Jun ":"01.06.",
    "Jul ":"01.07.",
    "Aug ":"01.08.",
    "Sept ":"01.09.",
    "Sep ":"01.09.",
    "Okt ":"01.10.",
    "Nov ":"01.11.",
    "Dez ":"01.12."
}

excel_dataset_pats.date_first_diagnosis_brain_met = excel_dataset_pats.date_first_diagnosis_brain_met.astype("str")

for month in months.keys():
    excel_dataset_pats.date_first_diagnosis_brain_met = excel_dataset_pats.date_first_diagnosis_brain_met.str.replace(month, months[month])


print(excel_dataset_pats.date_first_diagnosis_brain_met)

excel_dataset_pats.date_first_diagnosis_brain_met = pd.to_datetime(excel_dataset_pats.date_first_diagnosis_brain_met, errors="raise", format="mixed")
#excel_dataset_pats.birthdate = pd.to_datetime(excel_dataset_pats.birthdate, errors="raise", format="mixed")

0               01.09.2003
1               01.04.2005
2               01.04.2005
4               01.05.2005
5               01.05.2005
              ...         
735    2023-10-01 00:00:00
736    2023-10-01 00:00:00
740    2023-10-01 00:00:00
741    2023-10-01 00:00:00
744    2023-11-01 00:00:00
Name: date_first_diagnosis_brain_met, Length: 471, dtype: object


for some reason patient `1616767` had for `date_first_diagnosis_brainmet` the following value: `04.11.5240`\
changing to `01.11.2022` as this was more reasonable considering that the MRI image was taken on the `15.12.2022` and "neighboring" also had `date_first_diagnosis_brainmet` of `November`oder `Dezember 2022`

#### men

In [22]:
men_age_first_brainmet_dict = {}
for index, row in excel_dataset_pats[excel_dataset_pats.ID.isin(men_ids)].iterrows():
    id = row["ID"]

    # calculate the age of the patient when first diagnosed with tumor
    if not pd.isnull(row.date_first_diagnosis_brain_met):
        age = relativedelta(row.date_first_diagnosis_brain_met, row.birthdate).years
        men_age_first_brainmet_dict[id] = age
    else:
        #print(id)
        pass


men_age_first_brainmet_dict = list(men_age_first_brainmet_dict.values())

mean_ages_first_brainmet_men = np.mean(men_age_first_brainmet_dict)
median_ages_first_brainmet_men = np.median(men_age_first_brainmet_dict)
std_ages_first_brainmet_men = np.std(men_age_first_brainmet_dict)

print("{:^50}".format("ages of men first diagnosed with brainmetastases"))
print("{:12s}={:>6d}".format("total: n", len(men_age_first_brainmet_dict)))
print("{:12s}={:>9.2f}".format("mean: m", mean_ages_first_brainmet_men))
print("{:12s}={:>9.2f}".format("median: med", median_ages_first_brainmet_men))
print("{:12s}={:>9.2f}".format("std", std_ages_first_brainmet_men))

 ages of men first diagnosed with brainmetastases 
total: n    =   255
mean: m     =    63.37
median: med =    65.00
std         =    11.73


#### women

In [23]:
women_age_first_brainmet_dict = {}
for index, row in excel_dataset_pats[excel_dataset_pats.ID.isin(women_ids)].iterrows():
    id = row["ID"]

    # calculate the age of the patient when first diagnosed with tumor
    if not pd.isnull(row.date_first_diagnosis_brain_met):
        age = relativedelta(row.date_first_diagnosis_brain_met, row.birthdate).years
        women_age_first_brainmet_dict[id] = age
    else:
        #print(id)
        pass


women_age_first_brainmet_dict = list(women_age_first_brainmet_dict.values())

mean_ages_first_brainmet_women = np.mean(women_age_first_brainmet_dict)
median_ages_first_brainmet_women = np.median(women_age_first_brainmet_dict)
std_ages_first_brainmet_women = np.std(women_age_first_brainmet_dict)

print("{:^50}".format("ages of men first diagnosed with brainmetastases"))
print("{:12s}={:>6d}".format("total: n", len(women_age_first_brainmet_dict)))
print("{:12s}={:>9.2f}".format("mean: m", mean_ages_first_brainmet_women))
print("{:12s}={:>9.2f}".format("median: med", median_ages_first_brainmet_women))
print("{:12s}={:>9.2f}".format("std", std_ages_first_brainmet_women))

 ages of men first diagnosed with brainmetastases 
total: n    =   215
mean: m     =    60.15
median: med =    61.00
std         =    12.11


### OP / Chemotherapy / Radiatio / Immunotherapy

##### calculate values

In [24]:
# get value counts of OP, Chemo, Radiotio, Immunotherapy
# Get combindations of them
#

# men
op_men = excel_dataset_pats.OP[excel_dataset_pats.ID.isin(men_ids)]#.dropna()
chemo_men = excel_dataset_pats.Chemotherapie[excel_dataset_pats.ID.isin(men_ids)]#.dropna()
radiatio_men = excel_dataset_pats.Radiatio[excel_dataset_pats.ID.isin(men_ids)]#.dropna()
immun_men = excel_dataset_pats.Immuntherapie[excel_dataset_pats.ID.isin(men_ids)]#.dropna()

# women
op_women = excel_dataset_pats.OP[excel_dataset_pats.ID.isin(women_ids)]#.dropna()
chemo_women = excel_dataset_pats.Chemotherapie[excel_dataset_pats.ID.isin(women_ids)]#.dropna()
radiatio_women = excel_dataset_pats.Radiatio[excel_dataset_pats.ID.isin(women_ids)]#.dropna()
immun_women = excel_dataset_pats.Immuntherapie[excel_dataset_pats.ID.isin(women_ids)]#.dropna()

##### display results

In [25]:
print("{:^23}".format("therapy men"))
print("{:14s}={:>5d}/{:>3d} ({:>2.2%})".format("OP",
                                               len(op_men[excel_dataset_pats.OP == 1.0]),
                                               len(op_men),
                                               percentage(len(op_men[excel_dataset_pats.OP == 1.0]), len(op_men))))
print("{:14s}={:>5d}/{:>3d} ({:>2.2%})".format("Chemotherapy",
                                               len(chemo_men[excel_dataset_pats.Chemotherapie == 1.0]),
                                               len(chemo_men),
                                               percentage(len(chemo_men[excel_dataset_pats.Chemotherapie == 1.0]), len(chemo_men))))
print("{:14s}={:>5d}/{:>3d} ({:>2.2%})".format("Radiation",
                                               len(radiatio_men[excel_dataset_pats.Radiatio == 1.0]),
                                               len(radiatio_men),
                                               percentage(len(radiatio_men[excel_dataset_pats.Radiatio == 1.0]), len(radiatio_men))))
print("{:14s}={:>5d}/{:>3d} ({:>2.2%})".format("Immunotherapy",
                                               len(immun_men[excel_dataset_pats.Immuntherapie == 1.0]),
                                               len(immun_men),
                                               percentage(len(immun_men[excel_dataset_pats.Immuntherapie == 1.0]), len(immun_men))))
print()

print("{:^23}".format("therapy women"))
print("{:14s}={:>5d}/{:>3d} ({:>2.2%})".format("OP",
                                               len(op_women[excel_dataset_pats.OP == 1.0]),
                                               len(op_women),
                                               percentage(len(op_women[excel_dataset_pats.OP == 1.0]), len(op_women))))
print("{:14s}={:>5d}/{:>3d} ({:>2.2%})".format("Chemotherapy",
                                               len(chemo_women[excel_dataset_pats.Chemotherapie == 1.0]),
                                               len(chemo_women),
                                               percentage(len(chemo_women[excel_dataset_pats.Chemotherapie == 1.0]), len(chemo_women))))
print("{:14s}={:>5d}/{:>3d} ({:>2.2%})".format("Radiation",
                                               len(radiatio_women[excel_dataset_pats.Radiatio == 1.0]),
                                               len(radiatio_women),
                                               percentage(len(radiatio_women[excel_dataset_pats.Radiatio == 1.0]), len(radiatio_women))))
print("{:14s}={:>5d}/{:>3d} ({:>2.2%})".format("Immunotherapy",
                                               len(immun_women[excel_dataset_pats.Immuntherapie == 1.0]),
                                               len(immun_women),
                                               percentage(len(immun_women[excel_dataset_pats.Immuntherapie == 1.0]), len(immun_women))))

      therapy men      
OP            =  186/255 (72.94%)
Chemotherapy  =  144/255 (56.47%)
Radiation     =  184/255 (72.16%)
Immunotherapy =  111/255 (43.53%)

     therapy women     
OP            =  149/216 (68.98%)
Chemotherapy  =  127/216 (58.80%)
Radiation     =  148/216 (68.52%)
Immunotherapy =   76/216 (35.19%)


### Height / Weight / ECOG

In [26]:
# get vlaues of height, weight, ECOG
# if both height and weight are available
# calculate BMI if possible


# Height
height_men = excel_dataset_pats[excel_dataset_pats.ID.isin(men_ids)]["Größe (cm)"].dropna()
mean_height_men = np.mean(height_men)
median_height_men = np.median(height_men)
std_height_men = np.std(height_men)

height_women = excel_dataset_pats[excel_dataset_pats.ID.isin(women_ids)]["Größe (cm)"].dropna()
mean_height_women = np.mean(height_women)
median_height_women = np.median(height_women)
std_height_women = np.std(height_women)


# Weight
weight_men = excel_dataset_pats[excel_dataset_pats.ID.isin(men_ids)]["Gewicht (kg)"].dropna()
mean_weight_men = np.mean(weight_men)
median_weight_men = np.median(weight_men)
std_weight_men = np.std(weight_men)

weight_women = excel_dataset_pats[excel_dataset_pats.ID.isin(women_ids)]["Gewicht (kg)"].dropna()
mean_weight_women = np.mean(weight_women)
median_weight_women = np.median(weight_women)
std_weight_women = np.std(weight_women)


# BMI
# check if both values are available
# MEN
bmi_men = {}
for index, row in excel_dataset_pats[excel_dataset_pats.ID.isin(men_ids)].iterrows():
    id = row["ID"]
    weight = row["Gewicht (kg)"]
    height = row["Größe (cm)"]

    if not pd.isna(height) and not pd.isna(weight):
        bmi = float(weight) / np.square(float(height / 100))
        bmi_men[id] = bmi

bmi_men = list(bmi_men.values())
mean_bmi_men = np.mean(bmi_men)
median_bmi_men = np.median(bmi_men)
std_bmi_men = np.std(bmi_men)

# WOMEN
bmi_women = {}
dict_bmi_women = {}
counter = 0
for index, row in excel_dataset_pats[excel_dataset_pats.ID.isin(women_ids)].iterrows():
    id = row["ID"]
    weight = row["Gewicht (kg)"]
    height = row["Größe (cm)"]

    if not pd.isna(height) and not pd.isna(weight):
        bmi = float(weight) / np.square(float(height / 100))
        bmi_women[id] = bmi
        dict_bmi_women[counter] = bmi
        counter += 1

bmi_women = list(bmi_women.values())
mean_bmi_women = np.mean(bmi_women)
median_bmi_women = np.median(bmi_women)
std_bmi_women = np.std(bmi_women)


# ECOG
ecog_men = excel_dataset_pats[excel_dataset_pats.ID.isin(men_ids)].ECOG.dropna()
mean_ecog_men = np.mean(ecog_men)
median_ecog_men = np.median(ecog_men)
std_ecog_men = np.std(ecog_men)

ecog_women = excel_dataset_pats[excel_dataset_pats.ID.isin(women_ids)].ECOG.dropna()
mean_ecog_women = np.mean(ecog_women)
median_ecog_women = np.median(ecog_women)
std_ecog_women = np.std(ecog_women)

#### Height

In [27]:
print("{:^20}".format("height men"))
print("{:12s}={:>6d}".format("total: n", len(height_men)))
print("{:12s}={:>9.2f}".format("mean: m", mean_height_men))
print("{:12s}={:>9.2f}".format("median: med", median_height_men))
print("{:12s}={:>9.2f}".format("std", std_height_men))
print()

print("{:^20}".format("height women"))
print("{:12s}={:>6d}".format("total: n", len(height_women)))
print("{:12s}={:>9.2f}".format("mean: m", mean_height_women))
print("{:12s}={:>9.2f}".format("median: med", median_height_women))
print("{:12s}={:>9.2f}".format("std", std_height_women))

     height men     
total: n    =   179
mean: m     =   176.05
median: med =   176.00
std         =     6.68

    height women    
total: n    =   157
mean: m     =   164.29
median: med =   165.00
std         =     7.36


#### Weight

In [28]:
print("{:^20}".format("weight men"))
print("{:12s}={:>6d}".format("total: n", len(weight_men)))
print("{:12s}={:>9.2f}".format("mean: m", mean_weight_men))
print("{:12s}={:>9.2f}".format("median: med", median_weight_men))
print("{:12s}={:>9.2f}".format("std", std_weight_men))
print()

print("{:^20}".format("weight women"))
print("{:12s}={:>6d}".format("total: n", len(weight_women)))
print("{:12s}={:>9.2f}".format("mean: m", mean_weight_women))
print("{:12s}={:>9.2f}".format("median: med", median_weight_women))
print("{:12s}={:>9.2f}".format("std", std_weight_women))

     weight men     
total: n    =   180
mean: m     =    80.88
median: med =    80.00
std         =    14.58

    weight women    
total: n    =   168
mean: m     =    68.14
median: med =    68.00
std         =    14.27


#### BMI

In [29]:
print("{:^20}".format("bmi men"))
print("{:12s}={:>6d}".format("total: n", len(bmi_men)))
print("{:12s}={:>9.2f}".format("mean: m", mean_bmi_men))
print("{:12s}={:>9.2f}".format("median: med", median_bmi_men))
print("{:12s}={:>9.2f}".format("std", std_bmi_men))
print()

print("{:^20}".format("bmi women"))
print("{:12s}={:>6d}".format("total: n", len(bmi_women)))
print("{:12s}={:>9.2f}".format("mean: m", mean_bmi_women))
print("{:12s}={:>9.2f}".format("median: med", median_bmi_women))
print("{:12s}={:>9.2f}".format("std", std_bmi_women))

      bmi men       
total: n    =   178
mean: m     =    26.11
median: med =    25.93
std         =     4.57

     bmi women      
total: n    =   157
mean: m     =    25.22
median: med =    24.97
std         =     5.05


#### ECOG

In [30]:
print("{:^20}".format("ECOG men"))
print("{:12s}={:>6d}".format("total: n", len(ecog_men)))
print("{:12s}={:>9.2f}".format("mean: m", mean_ecog_men))
print("{:12s}={:>9.2f}".format("median: med", median_ecog_men))
print("{:12s}={:>9.2f}".format("std", std_ecog_men))

# ECOGs men
print("{:12s}:{:>5d}/{:>3d} ({:>2.2%})".format("ECOG 0", list(ecog_men).count(0.0), len(ecog_men), percentage(list(ecog_men).count(0.0), len(ecog_men))))
print("{:12s}:{:>5d}/{:>3d} ({:>2.2%})".format("ECOG 1", list(ecog_men).count(1.0), len(ecog_men), percentage(list(ecog_men).count(1.0), len(ecog_men))))
print("{:12s}:{:>5d}/{:>3d} ({:>2.2%})".format("ECOG 2", list(ecog_men).count(2.0), len(ecog_men), percentage(list(ecog_men).count(2.0), len(ecog_men))))
print("{:12s}:{:>5d}/{:>3d} ({:>2.2%})".format("ECOG 3", list(ecog_men).count(3.0), len(ecog_men), percentage(list(ecog_men).count(3.0), len(ecog_men))))
print("{:12s}:{:>5d}/{:>3d} ({:>2.2%})".format("ECOG 4", list(ecog_men).count(4.0), len(ecog_men), percentage(list(ecog_men).count(4.0), len(ecog_men))))
print("{:12s}:{:>5d}/{:>3d} ({:>2.2%})".format("ECOG 5", list(ecog_men).count(5.0), len(ecog_men), percentage(list(ecog_men).count(5.0), len(ecog_men))))
print()

print("{:^20}".format("ECOG women"))
print("{:12s}={:>6d}".format("total: n", len(ecog_women)))
print("{:12s}={:>9.2f}".format("mean: m", mean_ecog_women))
print("{:12s}={:>9.2f}".format("median: med", median_ecog_women))
print("{:12s}={:>9.2f}".format("std", std_ecog_women))

# ECOGs women
print("{:12s}:{:>5d}/{:>3d} ({:>2.2%})".format("ECOG 0", list(ecog_women).count(0.0), len(ecog_women), percentage(list(ecog_women).count(0.0), len(ecog_women))))
print("{:12s}:{:>5d}/{:>3d} ({:>2.2%})".format("ECOG 1", list(ecog_women).count(1.0), len(ecog_women), percentage(list(ecog_women).count(1.0), len(ecog_women))))
print("{:12s}:{:>5d}/{:>3d} ({:>2.2%})".format("ECOG 2", list(ecog_women).count(2.0), len(ecog_women), percentage(list(ecog_women).count(2.0), len(ecog_women))))
print("{:12s}:{:>5d}/{:>3d} ({:>2.2%})".format("ECOG 3", list(ecog_women).count(3.0), len(ecog_women), percentage(list(ecog_women).count(3.0), len(ecog_women))))
print("{:12s}:{:>5d}/{:>3d} ({:>2.2%})".format("ECOG 4", list(ecog_women).count(4.0), len(ecog_women), percentage(list(ecog_women).count(4.0), len(ecog_women))))
print("{:12s}:{:>5d}/{:>3d} ({:>2.2%})".format("ECOG 5", list(ecog_women).count(5.0), len(ecog_women), percentage(list(ecog_women).count(5.0), len(ecog_women))))

      ECOG men      
total: n    =   168
mean: m     =     0.94
median: med =     1.00
std         =     0.97
ECOG 0      :   69/168 (41.07%)
ECOG 1      :   55/168 (32.74%)
ECOG 2      :   30/168 (17.86%)
ECOG 3      :   13/168 (7.74%)
ECOG 4      :    1/168 (0.60%)
ECOG 5      :    0/168 (0.00%)

     ECOG women     
total: n    =   163
mean: m     =     1.03
median: med =     1.00
std         =     1.01
ECOG 0      :   60/163 (36.81%)
ECOG 1      :   55/163 (33.74%)
ECOG 2      :   34/163 (20.86%)
ECOG 3      :   11/163 (6.75%)
ECOG 4      :    3/163 (1.84%)
ECOG 5      :    0/163 (0.00%)


### Resektionstatus

In [31]:
#Resektionsstatus (0 = Komplettresektion; 1 = Rest/weitere Hirnfiliae, 2 = keine Resektion, 3 = unklar)

# MEN
men_resections = excel_dataset_pats["Resektionsstatus (0 = Komplettresektion; 1 = Rest/weitere Hirnfiliae, 2 = keine Resektion, 3 = unklar)"][excel_dataset_pats.ID.isin(men_ids)].dropna()

total_r_men = len(men_resections)
r0_men = 0
r1_men = 0
noR_men = 0

for value, amount in men_resections.value_counts().items():
    if value == 0.0:
        r0_men = amount
    elif value == 1.0:
        r1_men = amount
    elif value == 2.0:
        noR_men = amount
    elif value == 3.0:
        total_r_men -= amount
    else:
        print(f"error, unknown value {value}, {amount}")

# WOMEN
women_resections = excel_dataset_pats["Resektionsstatus (0 = Komplettresektion; 1 = Rest/weitere Hirnfiliae, 2 = keine Resektion, 3 = unklar)"][excel_dataset_pats.ID.isin(women_ids)].dropna()

total_r_women = len(women_resections)
r0_women = 0
r1_women = 0
noR_women = 0

for value, amount in women_resections.value_counts().items():
    if value == 0.0:
        r0_women = amount
    elif value == 1.0:
        r1_women = amount
    elif value == 2.0:
        noR_women = amount
    elif value == 3.0:
        total_r_women -= amount
    else:
        print(f"error, unknown value {value}, {amount}")

In [32]:
print("{:^35}".format("resection status men"))
print("{:20s}:{:>5d}/{:>3d} ({:>2.2%})".format("complete resection", r0_men, total_r_men, percentage(r0_men, total_r_men)))
print("{:20s}:{:>5d}/{:>3d} ({:>2.2%})".format("partial resection", r1_men, total_r_men, percentage(r1_men, total_r_men)))
print("{:20s}:{:>5d}/{:>3d} ({:>2.2%})".format("no resection", noR_men, total_r_men, percentage(noR_men, total_r_men)))

print()

print("{:^35}".format("resection status women"))
print("{:20s}:{:>5d}/{:>3d} ({:>2.2%})".format("complete resection", r0_women, total_r_women, percentage(r0_women, total_r_women)))
print("{:20s}:{:>5d}/{:>3d} ({:>2.2%})".format("partial resection", r1_women, total_r_women, percentage(r1_women, total_r_women)))
print("{:20s}:{:>5d}/{:>3d} ({:>2.2%})".format("no resection", noR_women, total_r_women, percentage(noR_women, total_r_women)))

       resection status men        
complete resection  :   51/242 (21.07%)
partial resection   :   98/242 (40.50%)
no resection        :   93/242 (38.43%)

      resection status women       
complete resection  :   36/203 (17.73%)
partial resection   :   73/203 (35.96%)
no resection        :   94/203 (46.31%)


### Relapsestatus

In [33]:
#im Verlauf Rezidiv der Hirnmetastase (0 = nicht bekannt; 1 = ja)

men_relapse = list(excel_dataset_pats["im Verlauf Rezidiv der Hirnmetastase (0 = nicht bekannt; 1 = ja)"][excel_dataset_pats.ID.isin(men_ids)].dropna())

women_relapse = list(excel_dataset_pats["im Verlauf Rezidiv der Hirnmetastase (0 = nicht bekannt; 1 = ja)"][excel_dataset_pats.ID.isin(women_ids)].dropna())


print("{:^32}".format("relpase status men"))
print("{:17s}:{:>5d}/{:>3d} ({:>2.2%})".format("relapse", men_relapse.count(1.0), len(men_ids), percentage(men_relapse.count(1.0), len(men_ids))))
print("{:17s}:{:>5d}/{:>3d} ({:>2.2%})".format("relapse unknown",
                                               len(men_ids) - men_relapse.count(1.0),
                                               len(men_ids),
                                               percentage(len(men_ids) - men_relapse.count(1.0), len(men_ids))))

print()

print("{:^32}".format("relpase status women"))
print("{:17s}:{:>5d}/{:>3d} ({:>2.2%})".format("relapse", women_relapse.count(1.0), len(women_ids), percentage(women_relapse.count(1.0), len(women_ids))))
print("{:17s}:{:>5d}/{:>3d} ({:>2.2%})".format("relapse unknown",
                                               len(women_ids) - women_relapse.count(1.0),
                                               len(women_ids),
                                               percentage(len(women_ids) - women_relapse.count(1.0), len(women_ids))))

       relpase status men       
relapse          :   56/255 (21.96%)
relapse unknown  :  199/255 (78.04%)

      relpase status women      
relapse          :   27/216 (12.50%)
relapse unknown  :  189/216 (87.50%)
